<a href="https://colab.research.google.com/github/VineetGargOfficials/Learning-PyTorch/blob/main/CNN_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import torchvision
from torchvision.transforms import transforms
import torch.nn.functional as F


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'CPU')
print(device)

cuda


Hyper-Parameters

In [11]:
learning_rate = 0.001
epochs = 4
batch_size = 4

Transform DataSet

In [12]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

Importing Dataset

In [13]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Create DataLoader

In [14]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

Define classes

In [15]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Creating Model

In [19]:
class MyCNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.layers = nn.Sequential(
        nn.Conv2d(3, 6, 5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(6, 16, 5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
      )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(16*5*5, 120),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(120, 84),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(84, 10)

        )

  def forward(self, X):
    X = self.layers(X)
    X = self.classifier(X)
    return X

Apply Model

In [23]:
model = MyCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Training LOOP

In [24]:
for epoch in range(epochs):
    running_loss = 0.0

    for batch_feature, batch_labels in train_dataloader:
        batch_feature = batch_feature.to(device)
        batch_labels = batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_feature)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

print('Finished Training')


Epoch [1/4], Loss: 2.2760
Epoch [2/4], Loss: 2.0189
Epoch [3/4], Loss: 1.8383
Epoch [4/4], Loss: 1.7110
Finished Training


Evaluate Model

In [25]:
model.eval()

MyCNN(
  (layers): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=400, out_features=120, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=120, out_features=84, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [27]:
total=0
correct = 0

with torch.no_grad():
  for batch_features, batch_labels in test_dataloader:
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    output = model(batch_features)
    _, predicted = torch.max(output.data, 1)

    total += batch_labels.size(0)
    correct += (predicted == batch_labels).sum().item()

print(f'Accuracy: {100 * correct / total}')

Accuracy: 41.97


In [29]:

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for features, labels in test_dataloader:
        features = features.to(device)
        labels = labels.to(device)

        outputs = model(features)
        _, predictions = torch.max(outputs, 1)

        for label, prediction in zip(labels, predictions):
            class_name = classes[label]
            if label == prediction:
                correct_pred[class_name] += 1
            total_pred[class_name] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class {classname:5s} is {accuracy:.2f}%')


Accuracy for class plane is 50.10%
Accuracy for class car   is 67.10%
Accuracy for class bird  is 21.00%
Accuracy for class cat   is 11.70%
Accuracy for class deer  is 24.70%
Accuracy for class dog   is 39.10%
Accuracy for class frog  is 52.40%
Accuracy for class horse is 59.10%
Accuracy for class ship  is 46.10%
Accuracy for class truck is 48.40%
